In [54]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup
import re

# All champion names

In [55]:
patch = "12.2.1"
ddragon = r.get(f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/champion.json").json()
champions = list(ddragon['data'].keys())
print(champions)

['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiyana', 'Quinn', 'Rakan', 'Rammus', 'RekSai', 'Rell', 'Renekt

In [56]:
corrections = {
    "AurelionSol": "Aurelion_Sol",
    "Chogath": "Cho'Gath",
    "DrMundo": "Dr._Mundo",
    "JarvanIV": "Jarvan_IV",
    "Kaisa": "Kai'Sa",
    "Khazix": "Kha'Zix",
    "KogMaw": "Kog'Maw",
    "Leblanc": "LeBlanc", #wiki links are case sensitive
    "LeeSin": "Lee_Sin",
    "MasterYi": "Master_Yi",
    "MissFortune": "Miss_Fortune",
    "MonkeyKing": "Wukong", #lol
    "RekSai": "Rek'Sai",
    "TahmKench": "Tahm_Kench",
    "TwistedFate": "Twisted_Fate",
    "Velkoz": "Vel'Koz",
    "XinZhao": "Xin_Zhao"
}

In [57]:
lol_champions = list(map(corrections.get, champions, champions))

### Testing scraping League of Legends wiki

In [77]:
rell = r.get(f"https://leagueoflegends.fandom.com/wiki/Ahri/LoL/Audio")

In [66]:
for i in lol_champions:
    champ = r.get(f"https://leagueoflegends.fandom.com/wiki/{i}/LoL/Audio")
    if champ.status_code == 404:
        print(i + " " + str(champ.status_code))
    soup = BeautifulSoup(champ.text, 'html.parser')
    unsorted_lines = list(soup.find('div', class_="mw-parser-output").find_all('i'))
    sorted_lines = list(map(parse_voice_line, unsorted_lines))

Vel'koz 404


In [58]:
def parse_voice_line(v_line):
    tag_removed = str(v_line)[3:-4]
    if ((tag_removed[0] == '"') and (tag_removed[-1] == '"')):
        return(tag_removed[1:-1],True)
    else:
        return(tag_removed,False)

In [91]:
def get_champion_voice_lines(df, champion_name):
    champ = r.get(f"https://leagueoflegends.fandom.com/wiki/{champion_name}/LoL/Audio")
    if champ.status_code == 404:
        print(i + " " + str(champ.status_code))
    no_trivia = re.sub(r'<span class="mw-headline" id="Trivia">.*', '', champ.text, flags=re.DOTALL).strip()
    soup = BeautifulSoup(no_trivia, 'html.parser')
    unsorted_lines = list(soup.find('div', class_="mw-parser-output").find_all('i'))
    
    sorted_lines = list(map(parse_voice_line, unsorted_lines))
    formatted_lines = pd.DataFrame([[champion_name,line[0],line[1]] for line in sorted_lines],
                                   columns=['champion', 'voice_line', 'is_spoken'])
    #print(df.head())
    df = pd.concat([df,formatted_lines])
    return df

In [92]:
all_voice = pd.DataFrame(columns=['champion', 'voice_line', 'is_spoken'])
for champion in lol_champions:
    all_voice = get_champion_voice_lines(all_voice, champion)
all_voice.to_csv("all_voice_lines.csv", index=False, encoding='utf-8')

#### Cleanup Voice Lines

In [93]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext
# https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string

In [95]:
all_voice = pd.DataFrame(pd.read_csv("all_voice_lines.csv"))
print(all_voice.head())
print(len(all_voice))
all_voice = all_voice.dropna()
all_voice.drop(all_voice[all_voice.voice_line.str.endswith(".ogg")].index, inplace = True)
all_voice.drop_duplicates(subset ="voice_line",
                     keep = "first", inplace = True)
all_voice.voice_line = all_voice.voice_line.map(cleanhtml)
print(len(all_voice))
all_voice.to_csv("clean_voice_lines.csv")

  champion                                         voice_line  is_spoken
0   Aatrox             Now, hear the silence of annihilation!       True
1   Aatrox                                   Let me end this!       True
2   Aatrox  Pantheon, I see you! Like a lamprey you attach...       True
3   Aatrox  Pantheon! I will crush your godhood, I will ev...       True
4   Aatrox  Targon sends an aspect against me? Prepare you...       True
39565
28904


In [103]:
all_voice.champion.value_counts()

Kayn        864
Rakan       845
Lux         813
Ezreal      777
Ahri        647
           ... 
Amumu        12
Singed       12
Kassadin      9
Shaco         9
Bard          6
Name: champion, Length: 158, dtype: int64